In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os, sys, re
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))), "lib/python"))

import numpy as np

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader


In [ ]:
# service options
delimiter = '/'

## configuration options
data_path = '/home/cosmonaut/dev/pic/picopic/'

if os.path.isfile(data_path) or os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, verbose=True)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

time_range=[reader.meta.time[0], reader.meta.time[1]]
radius = 0.02
longitude = 0.02
ylim = None

cmap='terrain'
image_interpolation = 'nearest'

autoselect = True
show_grid=False
aspect='equal'

guess_number_ticks=20

cmp = 'E/r'
specie = None # simply init with none. Replace with required specie for specied grids
probe_path = None

if specie:
    probe_path = cmp + delimiter + specie
else:
    probe_path = cmp

deviation = 0

x_axis_label = r'$\mathit{t (s)}$'
y_axis_label = r"$\mathit{{Amlitude {}}}$".format(re.sub(delimiter, '_', cmp))

plot_name = r"$\mathbf{{Time\enspace Series\enspace Data\enspace for\enspace {x}}}$".format(x=probe_path)

In [ ]:
start_frame = None
end_frame = None
row_number = reader.meta.get_row_by_radius(radius)
col_number = reader.meta.get_col_by_longitude(longitude)

size = []
if deviation > 0:
    size = [row_number - deviation, col_number - deviation, row_number + deviation, col_number + deviation]

# get data
data = []

if deviation == 0:
    for probe in reader.meta.probes:
        if ( probe.shape == 'dot' 
            and probe.component == cmp
            and probe.size[0] == row_number
            and probe.size[1] == col_number ):
            start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            data = reader.dot_range(probe_path, row_number, col_number, start_frame, end_frame)
                
    if len(data) == 0 and autoselect:
        for probe in reader.meta.probes:
            if ( probe.shape == 'rec'
                and probe.component == cmp
                and probe.size[0] <= row_number
                and probe.size[1] <= col_number
                and probe.size[2] >= row_number
                and probe.size[3] >= col_number ):
                start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], probe.schedule)
                end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], probe.schedule)
                shape = [probe.size[0], probe.size[1], probe.size[2], probe.size[3]]
                data = reader.dot_rec_range(probe_path, row_number, col_number, shape, start_frame, end_frame)            

else:
    for probe in reader.meta.probes:
        if ( probe.shape == 'rec'
            and probe.component == cmp
            and probe.size[0] == size[0]
            and probe.size[1] == size[1]
            and probe.size[2] == size[2]
            and probe.size[3] == size[3] ):
            start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], probe.schedule)
            end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], probe.schedule)
            data = reader.rec_range(probe_path, size, start_frame, end_frame)

    # try bigger frames, if autoselect enabled
    if len(data) == 0 and autoselect:
        for probe in reader.meta.probes:
            if ( probe.shape == 'rec'
                and probe.component == cmp
                and probe.size[0] <= size[0]
                and probe.size[1] <= size[1]
                and probe.size[2] >= size[2]
                and probe.size[3] >= size[3] ):
                start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], probe.schedule)
                end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], probe.schedule)
                data = reader.rec_range(probe_path, probe.size, start_frame, end_frame)# [size[0]:size[2], size[1]:size[3]]

In [ ]:
# find mean value at corresponding square
if deviation > 0:
    data_new = []
    lvar = 0
    count = len(data[0]) * len(data[0,0])

    for ts in data:
        for i in ts:
            for j in i:
                lvar += j
        data_new.append(lvar / count)
        lvar = 0
    data = data_new

In [ ]:
# define plot builder
# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=reader.meta.figure_color, 
                   fig_width=reader.meta.figure_width,
                   fig_height=reader.meta.figure_height, 
                   fig_dpi=reader.meta.figure_dpi,
                   font_family=reader.meta.figure_font_family,
                   font_name=reader.meta.figure_font_name,
                   font_size=reader.meta.figure_font_size,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=guess_number_ticks,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

In [ ]:
data_timeline = np.linspace(time_range[0], time_range[1], len(data))

# add subplots
sub_plot = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# set y-limits
if ylim is not None:
    plot.set_ylim(ylim)

# add images
sub_plot.plot(data_timeline, data)

plot.show()